In [1]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

In [2]:
# определям правильные ответы на тест
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [3]:
image = cv2.imread('images/test_01.png') # читаем картинку
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # конвертируем в ч/б
blurred = cv2.GaussianBlur(gray, (5, 5), 0) # применяем гауссовский фильтр
edged = cv2.Canny(blurred, 75, 200) # используем Canny алгоритм для определения границ

In [4]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE) # находим контуры на картинке
cnts = imutils.grab_contours(cnts)
docCnt = None

In [5]:
if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True) # сортиурем контура по размеру в убывающем порядке

    # итерируемся по контурам
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # если апроксимированный контур имеет 4 точки
        # значит мы нашли контур бумаги
        if len(approx) == 4:
            docCnt = approx
            break

In [6]:
# трансформируем изображение в вид "сверху"
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

In [7]:
# применим бинаризацию к изображению
thresh = cv2.threshold(warped, 0, 255,
    cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [8]:
# ищем контур на бинаризованной картинке
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

In [9]:
# ищем круглые контуры
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    # in order to label the contour as a question, region
    # should be sufficiently wide, sufficiently tall, and
    # have an aspect ratio approximately equal to 1
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

In [10]:
# сортируем конутры сверху-вниз
questionCnts = contours.sort_contours(questionCnts,
    method="top-to-bottom")[0]
correct = 0

In [11]:
# в вопросе 5 варинатов ответа, поэтому будем итерировать по 5 батчам
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None

    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # определим маску, которая будет выделять только закрашенный ответ
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the
        # bubble area
        # применим маску к картинке и посчитаем кол-во ненулевых пикселей в круге
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)

        # если кол-во не 0 пикселей больше, чем в остальных кругах
        # выбираем этот круг как ответ
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1

    cv2.drawContours(paper, [cnts[k]], -1, color, 3)

score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)

[INFO] score: 80.00%


32